# Informações gerais

## Tema
* Aplicação de encoder para RAG (*Retrieval Augmented Generation*)

## Integrantes
* Cesar Hideki Imai - 10402758
* João Victor Dallapé Madeira - 10400725
* David Varão Lima Bentes Pessoa - 1040264

# Setup

## Integração dos pacotes

O pacote `vectordb2` é usado para armazenar e recuperar textos usando técnicas de *chunking* (segmentação de texto), *embedding* (conversão de texto para vetores numéricos) e busca vetorial.

In [82]:
%pip install vectordb2

O pacote requests pode ser usado para recuperar o arquivo por meio de requisição em HTTP (esse pacote é opcional).

In [83]:
%pip install requests

O pacote `networkx` é usado para a criação, manipulação e representação de grafos.

In [84]:
%pip install networkx

Importação das bibliotecas

In [85]:
from vectordb import Memory
import requests
import networkx
import os # Será usado métodos para limpar o terminal para atualizar a interface em cada iteração do sistema
import time # Será usado método de espera para atualizar a interface gradualmente

## Classe Grafo

In [86]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 13 13:59:10 2023

@author: icalc
"""
class Grafo:
    TAM_MAX_DEFAULT = 100 # qtde de vértices máxima default
    # construtor da classe grafo
    def __init__(self, n=TAM_MAX_DEFAULT):
        self.n = n # número de vértices
        self.m = 0 # número de arestas
        # matriz de adjacência
        self.adj = [[0 for i in range(n)] for j in range(n)]

	# Insere uma aresta no Grafo tal que
	# v é adjacente a w
    def insereA(self, v, w):
        if self.adj[v][w] == 0:
            self.adj[v][w] = 1
            self.m+=1 # atualiza qtd arestas

# remove uma aresta v->w do Grafo
    def removeA(self, v, w):
        if(v == w):
            return
        # testa se temos a aresta
        if self.adj[v][w] == 1:
            self.adj[v][w] = 0
            self.m -= 1  # atualiza qtd arestas

	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                if self.adj[i][w] == 1:
                    print(f"Adj[{i:2d},{w:2d}] = 1 ", end="")
                else:
                    print(f"Adj[{i:2d},{w:2d}] = 0 ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                if self.adj[i][w] == 1:
                    print(" 1 ", end="")
                else:
                    print(" 0 ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

    def leGrafo(self, path):
        with open(path, 'r',encoding='utf-8') as arquivo:
            n_palavras = int(arquivo.readline())

            palavras = []
            for i in range(n_palavras):
                palavras.append(arquivo.readline().strip())
        print(palavras)
        print(n_palavras)

## Classe GrafoND (Grafo não-direcionado)

In [87]:
# Grafo como uma matriz de adjacência não-direcionado
class GrafoND(Grafo): # Ex 7
    def __init__(self, n):
        super().__init__(n)

    def insereA(self, v, w):
        if(v == w):
            return
        if self.adj[v][w] == 0:
            self.adj[v][w] = 1
            self.adj[w][v] = 1
            self.m += 1  # atualiza qtd arestas

# remove uma aresta v->w do Grafo
    def removeA(self, v, w):
        if(v == w):
            return
        # testa se temos a aresta
        if self.adj[v][w] == 1:
            self.adj[v][w] = 0
            self.adj[w][v] = 0
            self.m -= 1  # atualiza qtd arestas

    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(int(self.n)):
            for w in range(self.n):
                print(f"Adj[{i:2d},{w:2d}] = {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

    #Verifica se o grafo é completo Ex 10)
    def ehCompleto(self):
        if((self.n ** 2 - self.n)/ 2 == self.m):
            return "O grafo é completo"
        else:
            return "O grafo não é completo"

    def conexidade(self): # Ex 13
        if(self.n > 1 and self.m == 0):
            return "O grafo não é conexo"
        passou = [0]
        for i in range(self.n):
            if(not i in passou):
                return "O grafo não é conexo"
            for j in range(self.n):
                if(i != j and self.adj[i][j] != 0 and not j in passou):
                    passou.append(j)
                    #print(passou)
        return "O grafo é conexo"

    def removeV(self, vertice): # Ex 24
        if(vertice >= self.n):
            return False
        for i in range(self.n-1):
            if(i >= vertice): # Substitui as conexões do vértice a ser retirado e
                              # os vértices posteriores a ele com as conexões do próximo vértice
                self.adj[i] = self.adj[i+1]
            self.removeA(i,vertice)
            self.adj[i].pop(vertice) # Remove o vértice escolhido da linha da matriz
        self.adj.pop() # Remove a última linha da matriz
        self.n -= 1
        return True

## Classe GrafoNDR (Grafo não-direcionado rotulado)

In [88]:
# Grafo como uma matriz de adjacência não-direcionado rotulado
class GrafoNDR(GrafoND): # Ex 8
# Não bota o init, vai bugar a classe

    def insereA(self, v, w, p):
        if(v == w):
            return
        if self.adj[v][w] == 0:
            self.adj[v][w] = p
            self.adj[w][v] = p
            self.m += 1  # atualiza qtd arestas

    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f"Adj[{i:2d},{w:2d}] = {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f" {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

## Classe Memory

Aqui é utilizado a biblioteca VectorDB para criar uma memória virtual.

```
memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
```

- `chunking_strategy`: Define a estratégia de fragmentação dos dados. No modo "sliding_window", os dados são divididos em *chunks* (pedaços de texto) de tamanho fixo.

- `window_size`: Define a quantidade de palavras que um *chunk* representa. Neste caso, cada *chunk* representa uma palavra.

- `overlap`: Define quantos elementos de sobreposição existirão entre os *chunks* adjacentes. Neste caso, não haverá sobreposição já que as palavras usadas não formam frases, logo são independentes uma das outras.

# Métodos

## 1. Ler dados

### Aquisição dos dados

Os dados do documento são importados e guardados na variável `dados`.

In [103]:
def leArquivoHTTP():
  arquivo = requests.get('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/palavras.txt').text

  palavras = arquivo.split()
  n_palavras = int(palavras.pop(0))
  dados = {
      "n_palavras": n_palavras,
      "palavras": palavras
  }

  return dados

In [105]:
def leArquivo(origem):
  with open(origem, 'r', encoding='utf-8') as arquivo:
    n_palavras = int(arquivo.readline())

    palavras = [""] * n_palavras
    for i in range(n_palavras):
      palavras[i] = arquivo.readline().strip()

  dados = {
      "n_palavras": n_palavras,
      "palavras": palavras
  }

  return dados

In [106]:
#d = leArquivoHTTP()
d = leArquivo("./Projeto/palavras.txt")
print(d)

{'n_palavras': 50, 'palavras': ['Ecossistema', 'Sustentabilidade', 'Biodiversidade', 'Reciclagem', 'Conservação', 'Poluição', 'Desmatamento', 'Reflorestamento', 'Erosão', 'Compostagem', 'Biodegradável', 'Emissões', 'Pegada', 'Recursos', 'Preservação', 'Ecologia', 'Habitat', 'Fauna', 'Flora', 'Agroecologia', 'Bioma', 'Ciclo', 'Desenvolvimento', 'Economia', 'Efluentes', 'Gestão', 'Impacto', 'Mata', 'Amazônia', 'Cerrado', 'Pantanal', 'Biotecnologia', 'Agrofloresta', 'Agricultura', 'Aquicultura', 'Biocombustível', 'Solar', 'Eólica', 'Hidrelétrica', 'Resíduos', 'Saneamento', 'Tratamento', 'Uso', 'Zona', 'Proteção', 'Ambiental', 'Clima', 'Solo', 'Água', 'Floresta']}


### Embedding

Cada palavra é convertida para um vetor numérico e guardada na memória.

In [92]:
def embedding(memoria, palavras, n_palavras):
  for i in range(n_palavras):
    memoria.save(palavras[i])

### Busca vetorial

Quanto menor é a distância, maior é a proximidade semântica.

In [93]:
def buscaVetorial(memoria, palavra):
  busca = memoria.search(palavra, top_n=6)
  return busca

A palavra mais próxima armazenada na memória é ela mesma, portanto para encontrar as outras cinco palavras mais próximas foi recuperado as palavras de índice 1 até 6.

In [99]:
m = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
n = d["n_palavras"]
p = d["palavras"]

embedding(m, p, n)

b = buscaVetorial(m, "Biodiversidade")
print(b)
print(f"\n\nBusca: Biodiversidade\n")
for i in range(1,6):
  palavra = b[i]['chunk']
  distancia = b[i]['distance']
  print(f"Palavra: {palavra}\nDistância: {distancia:.2f}\n")

Initiliazing embeddings:  normal
OK.
[{'chunk': 'Biodiversidade', 'metadata': {}, 'distance': 0.0}, {'chunk': 'Sustentabilidade', 'metadata': {}, 'distance': 0.44970125}, {'chunk': 'Agroecologia', 'metadata': {}, 'distance': 0.492421}, {'chunk': 'Biotecnologia', 'metadata': {}, 'distance': 0.49487936}, {'chunk': 'Biodegradável', 'metadata': {}, 'distance': 0.5287854}, {'chunk': 'Bioma', 'metadata': {}, 'distance': 0.5513243}]


Busca: Biodiversidade

Palavra: Sustentabilidade
Distância: 0.45

Palavra: Agroecologia
Distância: 0.49

Palavra: Biotecnologia
Distância: 0.49

Palavra: Biodegradável
Distância: 0.53

Palavra: Bioma
Distância: 0.55



### Integração no grafo

In [ ]:
def integraGrafo(memoria, palavras, n_palavras):
  grafo = GrafoNDR(n_palavras)
  for i in range(n_palavras):
    busca = buscaVetorial(memoria, palavras[i])
    for j in range(1,6):
      palavra = busca[j]['chunk']
      distancia = busca[j]['distance']

## 2. Gravar dados

## 3. Inserir vértice

## 4. Inserir aresta

## 5. Remover vértice

## 6. Remover aresta

## 7. Mostrar conteúdo

## 8. Mostrar grafo

## 9. Apresentar conexidade do grafo e o reduzido

# Menu

In [96]:
memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
fim = False

while(fim == False):
    print(
'''
Menu:
    1) Ler dados do arquivo em python
    2) Gravar dados no arquivo grafo.txt
    3) Inserir vértice
    4) Inserir aresta
    5) Remove vértice
    6) Remove aresta
    7) Mostrar conteúdo do arquivo
    8) Mostrar grafo
    9) Apresentar a conexidade do grafo e o reduzido
    10) Encerrar a aplicação
''')
    choice = int(input())
    if choice == 1: # Lê grafo
        dados = leArquivoHTTP()
        #dados = leArquivo("palavras.txt")
        embedding(memoria, dados["palavras"], dados["n_palavras"])
        print("Grafo lido com sucesso!")

    elif choice == 2: # Grava dados no arquivo .txt
        print("Dados salvos com sucesso!")

    elif choice == 3: # Insere vértice
        print("Vértice inserido com sucesso!")

    elif choice == 4: # Insere aresta
        print("Arestas inseridas com sucesso!")

    elif choice == 5: # Remove vértice
        print("Vértice removido com sucesso!")

    elif choice == 6: # Remove varesta
        print("Aresta removida com sucesso!")

    elif choice == 7: # Imprime arquivo
        print("oi")

    elif choice == 8: # Exibe grafo
        print("oi")

    elif choice == 9: # Apresenta a conexidade do grafo e grafo reduzido
        print("oi")

    elif choice == 10: # Encerra
        fim = True
        print("Encerrando programa...")

    else:
        print("Opção inválida.")

    time.sleep(4) # Volta para o menu após 4 segundos

    if os.name == 'nt': # Limpa o terminal
        os.system('cls') # Caso o OS seja Windows
    else:
        os.system('clear') # Caso o OS seja Linux ou MacOS

Initiliazing embeddings:  normal
OK.

Menu:
    1) Ler dados do arquivo em python
    2) Gravar dados no arquivo grafo.txt
    3) Inserir vértice
    4) Inserir aresta
    5) Remove vértice
    6) Remove aresta
    7) Mostrar conteúdo do arquivo
    8) Mostrar grafo
    9) Apresentar a conexidade do grafo e o reduzido
    10) Encerrar a aplicação

1
Grafo lido com sucesso!

Menu:
    1) Ler dados do arquivo em python
    2) Gravar dados no arquivo grafo.txt
    3) Inserir vértice
    4) Inserir aresta
    5) Remove vértice
    6) Remove aresta
    7) Mostrar conteúdo do arquivo
    8) Mostrar grafo
    9) Apresentar a conexidade do grafo e o reduzido
    10) Encerrar a aplicação

10
Encerrando programa...
